In [2]:
from matplotlib.offsetbox import AnchoredOffsetbox, TextArea, HPacker, VPacker

%run basics_sensorSignal.py
%matplotlib

Using matplotlib backend: MacOSX


sammelsurium an verschiedenen funktionen

In [3]:
def henderson_hasselbalch(c_nh3, c_nh4, pKa=9.25):
    """ Returns the pH depending on the concentrations of NH3 and NH4+ """
    return pKa + np.log(c_nh3/c_nh4)

def henderson_nh3(pH, c_nh4, pKa=9.25):
    """ Returns the NH3 concentration depending on the pH and the concentrations of NH4+ """
    return c_nh4 * 10**(pH-pKa)

def total_ammonia(c_nh4, pH, pKa=9.25):
    """ Returns the total ammonia concentration depending on the pH and the 
    concentrations of NH4+ """
    if isinstance(pH, list):
        tan = [c_nh4 * (1+ 10**(ph_i - pKa)) for ph_i in ls_pH]  
    else:
        tan = c_nh4 * (1 + 10**(pH-pKa))
    return tan

def partial_concNH3(c_nh4, pH, pKa=9.25):
    """ Returns the percentage of NH4+ concentration depending on the pH """
    if isinstance(pH, list):
        numerator = [c_nh4 * 10**(ph_i - pKa) for ph_i in ls_pH]
        denominator = [c_nh4 * (1+10**(ph_i - pKa)) for ph_i in ls_pH]
        c_nh3 = [n/d for (n,d) in zip(numerator,denominator)]
    else:
        c_nh3 = (c_nh4 * (10**(pH-pKa))) / (c_nh4 * (1+10**(pH-pKa))) 
    return c_nh3

def int2pH(intF, int_max, pKa):
    #precheck - intensity out of range
    f_min = round(int_max * np.exp(-pKa)/ (1 + np.exp(-pKa)), 5)      # pH >= 0
    f_max = round(int_max * np.exp(14-pKa)/ (1 + np.exp(14-pKa)), 5)  # pH <= 0
    
    ls_ph = list()
    for f in intF:
        if f > f_min and f < f_max:
            c = np.log(f / (int_max - f))
            ls_ph.append(round(pKa + c, 5))
        elif f < f_min:
            ls_ph.append(0)
        else:
            ls_ph.append(14)
    return ls_ph

def Nernst_equation(ls_ph, E0, T=25):
    # global parameter
    n, R, F = 1, 8.314, 96485 # [J/mol*K], [C/mol]
    
    faq = 2.303* R*(T+273.15) / (n*F)
    # logarithmic of activity equals pH 
    return E0 - faq * ls_ph

def Nernst_equation_invert(E, E0, T=25):
    # global parameter
    n, R, F = 1, 8.314, 96485 # [J/mol*K], [C/mol]
    return (E0 - E/1000) / (2.303*R*(T+273.15)) *n*F

simulate NH3 vs NH4 system

In [4]:
pKa = 9.25                  # pKa of ammonia
c_nh4_GG = 2.3e-6           # concentration of ammonium ion at the pKa; in mol/L
tan = 2*c_nh4_GG
pH = np.linspace(0, 14, num=int(14/0.01+1))          # pH values 0-14

alpha = partial_concNH3(c_nh4=c_nh4_GG, pH=pH, pKa=9.25)
df_alpha = pd.DataFrame(alpha, index=pH, columns=['nh4'])
xnew = [round(i, 2) for i in df_alpha.index]
df_alpha.index = xnew

In [6]:
fs = 11

fig, ax = plt.subplots(figsize=(5,3))
fig.canvas.set_window_title('Ammonia vs. Ammonium')
ax.set_xlabel('pH', fontsize=fs), ax.set_ylabel('alpha [%]', fontsize=fs)

ax.plot(pH, alpha, color='#196E94', lw=1., label='NH$_4^+$')
ax.plot(pH, 1-alpha, color='#FFA02E', lw=1., label='NH$_3$')

ax.axvline(pKa, color='k', ls=':', lw=1.)
ax.axhline(0.5, color='k', ls=':', lw=1.)
ax.legend(loc='upper center', bbox_to_anchor=(1,0.9), frameon=True, fancybox=True, 
          fontsize=fs*0.7)

ax.set_xlim(0, 14)
sns.despine()
plt.tight_layout()

simulate experiment -  pH changes over time

In [7]:
# time intervals pH fluctuations - step function
t_plateau = 30                     # int, seconds for one pH value
pH_steps = list(np.arange(0, 15))  # steps of pH changes; linearly or randomly

random = input('Select: (1) Random pH gradient or (2) ramp function? >')
if random == '1' or random == 'random':
    np.random.shuffle(pH_steps)
elif random == '2' or random == 'ramp':
    pH_steps = pH_steps
else:
    raise ValueError('Choose between 1 and 2')

print('Targeted pH changes:', pH_steps)

#create time range
l = [[ph_i] * t_plateau for ph_i in pH_steps]
ls_pH = l[0]
lspH = [ls_pH.extend(l_i) for l_i in l[1:]]

#finalize target pH and concentrations
df_pH = pd.DataFrame(ls_pH, columns=['pH'])
for i in df_pH.index:
    df_pH.loc[i, 'alpha_nh4 [%]'] = df_alpha['nh4'].loc[df_pH.loc[i, 'pH']]
    df_pH.loc[i, 'alpha_nh3 [%]'] = 1-df_alpha['nh4'].loc[df_pH.loc[i, 'pH']]
df_pH.index.name = 'time / s'
ls_time = df_pH.index

Select: (1) Random pH gradient or (2) ramp function? >2
Targeted pH changes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [26]:
#pH sensor response
sig_max = 270       # maximal signal response at maximal NH4+ concentration in mV 
sig_bgd = 20        # background signal / offset in mV at 0M NH4+
tresp_pH = 20       # response time of pH sensor in seconds
para_pH = dict({'max conc': sig_max*1e-3, 'response time': tresp_pH, 
                'background signal': sig_bgd*1e-3})

# electrochemical pH sensor calibration
# compare: https://www.vernier.com/2018/04/06/the-theory-behind-ph-measurements/
dfSig_calib = pd.DataFrame(Nernst_equation(ls_ph=np.array(pH_steps), T=25, 
                                           E0=(sig_bgd+sig_max)/1000),#+sig_bgd/1000, 
                           index=pH_steps, columns=['signal / mV'])*1000
dfSig_calib.index.name = 'pH'

In [27]:
# calibration plot pH sensor
fig, ax = plt.subplots(figsize=(5,3))
fig.canvas.set_window_title('electrochemical pH sensor')
ax.set_xlabel('pH', fontsize=fs), ax.set_ylabel('Potential / mV', fontsize=fs)

ax.plot(dfSig_calib, lw=1., color='k')

sns.despine()
plt.tight_layout()

In [28]:
# pH changes -> target potential
dfSig_steps = pd.DataFrame(Nernst_equation(ls_ph=np.array(df_pH['pH'].to_numpy()), T=25, 
                                           E0=(sig_bgd+sig_max)/1000), #+sig_bgd/1000,
                           index=df_pH['pH'].index, columns=['target potential / mV'])*1000
dfpH_target = pd.concat([df_pH['pH'], dfSig_steps], axis=1)

# plot target pH and potential
fig, ax = plt.subplots(figsize=(5,3))
fig.canvas.set_window_title('pH sensor - target pH and potential')
ax1 = ax.twinx()
ax.set_xlabel('Time / s'), ax.set_ylabel('Potential / mV')
ax1.set_ylabel('pH', color='gray')
ax.plot(dfpH_target['target potential / mV'], color='k', ls='-.', lw=1.)
ax1.plot(dfpH_target['pH'], lw=1., ls=':', color='gray')
plt.subplots_adjust(left=0.15, right=0.89, bottom=0.16, top=0.92)

In [29]:
step = 0.01
# sensor response --> make a function out of it
dsig = dict()
for n in range(len(pH_steps)):
    if n == 0:
        y_1 = sig_bgd
    else:
        y_1 = dsig[n-1]['signal / mV'].to_numpy()[-1]
    sdiff = dfpH_target['target potential / mV'].loc[n*t_plateau]-y_1
    
    if n == 0:
        # 1st sensor response - always differnet
        t_sens = np.arange(0, t_plateau-1, step)
        y0 = _gompertz_curve_v1(x=t_sens, t90=tresp_pH, tau=1e-5, pstart=sig_bgd,
                                slope='increase', s_diff=sdiff)
        dfSig = pd.DataFrame(y0, index=t_sens, columns=['signal / mV'])
    else:
        # other sensor responses behave similar
        t_sens_ = np.arange(0, t_plateau+step, step)
          
        # different cases - decline or increase
        if sdiff < 0:
            y1 = _gompertz_curve_v1(x=t_sens_, t90=tresp_pH, tau=1e-5, pstart=y_1, 
                                    slope='decline', s_diff=np.abs(sdiff))
        else:
            y1 = _gompertz_curve_v1(x=t_sens_, t90=tresp_pH, tau=1e-5, pstart=y_1, 
                                    slope='increase', s_diff=np.abs(sdiff))
        t_sens1 = t_sens_+n*t_plateau+(n-1)*step-1
        dfSig = pd.DataFrame(y1, index=t_sens1, columns=['signal / mV'])
    dsig[n] = dfSig
    
sens_response = pd.concat(dsig)
xnew = sens_response.index.levels[1]
sens_response.index = xnew

#---------------------------------------------------------
# recalculation of pH values
y = Nernst_equation_invert(E=sens_response['signal / mV'], T=25, E0=(sig_bgd+sig_max)/1000)
dfph_re = pd.DataFrame([round(i, 2) for i in y], index=sens_response.index)
dfph_re.columns = ['pH recalc.']


In [30]:
figR, axR = plt.subplots(figsize=(5.8,4))
figR.canvas.set_window_title('pH sensor response')
axT = axR.twinx()
axR.set_xlabel('Time / s', fontsize=fs), axR.set_ylabel('Potential / mV', fontsize=fs)
axT.set_ylabel('pH', fontsize=fs, color='gray')
#..................
# target potential
lns1 = axR.plot(dfpH_target['target potential / mV'], color='k', ls='-.', lw=1., 
                label='target potential / mV')
# sensor response
lns2 = axR.plot(sens_response, color='#196E94', lw=1., label='sensor response / mV')

# re-calculated pH
lns3 = axT.plot(dfpH_target['pH'], lw=1., ls=':', color='gray', label='target pH')
lns4 = axT.plot(dfph_re, color='#FFA02E', lw=1., label='recalc. pH')

#..................
# combine legend
lns = lns1+lns2+lns3+lns4
labs = [l.get_label() for l in lns]
axR.legend(lns, labs, ncol=4, loc='upper center', bbox_to_anchor=(0.5, 1.1), 
           fontsize=fs*0.7)

#..................
plt.tight_layout()

In [32]:
# to be updated!!!!
# NH3 sensor is a measured signal as well!


In [31]:
# determine NH4+ concentration according to measured pH
nh4_pc = pd.DataFrame(df_alpha.loc[dfph_re['pH recalc.'].to_numpy()]['nh4'].to_numpy(),
                      index=dfph_re.index, columns=['nh4 % '])*100
nh3_pc = 100-nh4_pc
nh3_pc.columns = ['nh3 %']

# --------------------------------------------------
fig_re, ax_re = plt.subplots(figsize=(5,5), nrows=2, sharex=True)
axT = ax_re[0].twinx()

axT.set_ylabel('TAN', fontsize=fs)
ax_re[0].set_ylabel('pH', fontsize=fs, color='white')

ax_re[1].set_ylabel('pH', fontsize=fs)
ax_re[1].set_xlabel('Time / s', fontsize=fs)

fig_re.canvas.set_window_title('TAN simulation')
ax_re[0].plot(nh4_pc, lw=1., color='#196E94', label='NH$_4^+$')
ax_re[0].plot(nh3_pc, lw=1., color='#FFA02E', label='NH$_3$')
axT.plot(pd.concat([nh3_pc, nh4_pc], axis=1).sum(axis=1), lw=1., color='k', ls='-.',
              label='TAN')

ax_re[1].plot(dfph_re, color='k', lw=1.)

ax_re[0].spines['top'].set_visible(False), axT.spines['top'].set_visible(False)
ax_re[1].spines['top'].set_visible(False), ax_re[1].spines['right'].set_visible(False)

# labelling
ax_re[0].text(-100., 5, 'alpha [%]', color='k', rotation=90)
ax_re[0].text(-100., 47, 'NH$_4^+$', color='#196E94', rotation=90)
ax_re[0].text(-100., 65, 'and', color='k', rotation=90)
ax_re[0].text(-100., 83, 'NH$_3$', color='#FFA02E', rotation=90)

plt.tight_layout(pad=1)